In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
#Test 
url='https://www.moneycontrol.com/news/world/chinas-huawei-continues-rebound-with-strongest-earnings-growth-since-2019-12543141.html'
page=requests.get(url)
soup=BeautifulSoup(page.text,'html.parser')

In [3]:
soup.find('h2',class_='article_text').text.strip()

"China's Huawei continues rebound with strongest earnings growth since 2019"

In [4]:
soup.find_all('p')

[<p class="my_account">My Account</p>,
 <p>Follow us on:</p>,
 <p>Powered By <svg height="26" version="1.2" viewbox="0 0 71 26" width="71" xmlns="https://www.w3.org/2000/svg"><title>hBits - Fractional Ownership &amp; Investment In Commercial Real Estate</title><defs><image height="50" href="" id="img1" width="150"/></defs><use href="#img1" id="Signature Logo" transform="matrix(.473,0,0,.473,0,1)"></use></svg></p>,
 <p class="tit-txt2">Start generating passive income with up to 10% rental yield and an expected IRR of 18%</p>,
 <p class="MT5"><span class="mcp_btn_blue_tp" title="Invest Now">Invest Now</span></p>,
 <p>Powered By <svg fill="none" height="26" viewbox="0 0 71 26" width="71" xmlns="https://www.w3.org/2000/svg"><path d="M8.24577 6.69658C6.8123 6.69658 5.77336 7.09769 5.12895 7.89991C4.49769 8.68898 4.18207 9.77395 4.18207 11.1548C4.18207 11.8255 4.26097 12.4371 4.41879 12.9894C4.5766 13.5286 4.81332 13.9955 5.12895 14.39C5.44458 14.7845 5.83911 15.0936 6.31255 15.3172C6.78599 

In [5]:

div_tag = soup.find('div', class_='article_image')

news_paras = []

if div_tag:
    following_p_tags = div_tag.find_all_next('p')
    if following_p_tags:
        for p_tag in following_p_tags:
            news_paras.append(p_tag.text.strip())

In [6]:
news_paras

["China's Huawei Technologies saw its fastest growth in four years in 2023, with a rebound in its consumer segment and income from new businesses like smart car components accelerating its recovery from U.S. sanctions.",
 'Revenue rose 9.63% from a year earlier to 704.2 billion yuan ($97.48 billion), with the consumer business contributing most to that figure, growing 17.3% to 251.49 billion yuan.',
 'While Huawei did not break down the consumer figure, the segment includes its handset business, which saw a renaissance last year as the company returned to the mainstream 5G smartphones market with the Mate 60, apparently overcoming U.S. restrictions.',
 "From 2019, the U.S. has restricted Huawei's access to U.S. technology, accusing the company of being a security risk, which Huawei denies.",
 'Last year marked the third consecutive year of growth for the company after revenue plummeted by almost a third in 2021 when the company started to exhaust chip reserves, though revenue remains b

In [7]:
news_string = '\n'.join(news_paras)

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from heapq import nlargest  # Import nlargest function from heapq module

sentences = sent_tokenize(news_string)

stop_words = set(stopwords.words('english'))
word_tokens = [word.lower() for sentence in sentences for word in word_tokenize(sentence) if word.isalnum()]
filtered_tokens = [word for word in word_tokens if word not in stop_words]

word_freq = Counter(filtered_tokens)

sentence_scores = {}
for sentence in sentences:
    for word in word_tokenize(sentence.lower()):
        if word in word_freq:
            if len(sentence.split(' ')) < 30:  
                if sentence not in sentence_scores:
                    sentence_scores[sentence] = word_freq[word]
                else:
                    sentence_scores[sentence] += word_freq[word]

summary_sentences = nlargest(5, sentence_scores, key=sentence_scores.get)
summary = ' '.join(summary_sentences)

print("Summary:")
print(summary)


Summary:
Huawei's four-year-old smart car software and components business also saw major growth albeit from a lower comparison base, up 128.1% year on year to 4.7 billion yuan. Revenue rose 9.63% from a year earlier to 704.2 billion yuan ($97.48 billion), with the consumer business contributing most to that figure, growing 17.3% to 251.49 billion yuan. At a launch event last year, Meng Wangzhou, Huawei's CFO and the daughter of the company's founder, announced that Huawei was no longer in crisis mode. Last year Huawei announced it would spin off the smart car unit into a new company. While the company's core ICT infrastructure business remained stable, its cloud business grew by more than a fifth, generating revenue of 55.3 billion yuan.


In [8]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from heapq import nlargest
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 

news_string = '\n'.join(news_paras)

sentences = sent_tokenize(news_string)

stop_words = set(stopwords.words('english'))
word_tokens = [word.lower() for sentence in sentences for word in word_tokenize(sentence) if word.isalnum()]
filtered_tokens = [word for word in word_tokens if word not in stop_words]

word_freq = Counter(filtered_tokens)

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
tfidf_scores = tfidf_matrix.sum(axis=1)

sentence_scores = {sentence: score for sentence, score in zip(sentences, tfidf_scores)}

similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
similarity_graph = similarity_matrix - np.eye(similarity_matrix.shape[0])  
sentence_ranks = np.array(similarity_graph.sum(axis=1)).flatten()

for i, sentence in enumerate(sentences):
    sentence_scores[sentence] += sentence_ranks[i]

summary_sentences = nlargest(5, sentence_scores, key=sentence_scores.get)
summary = ' '.join(summary_sentences)

print("Summary:")
print(summary)


Summary:
Earlier this month, Richard Yu, the managing director and chairman of Huawei's smart car solutions, announced the unit would likely turn a profit from April after losing billions of yuan in the past year, local media reported. Last year marked the third consecutive year of growth for the company after revenue plummeted by almost a third in 2021 when the company started to exhaust chip reserves, though revenue remains below its 2020 peak of 891.3 billion yuan. While Huawei did not break down the consumer figure, the segment includes its handset business, which saw a renaissance last year as the company returned to the mainstream 5G smartphones market with the Mate 60, apparently overcoming U.S. restrictions. China's Huawei Technologies saw its fastest growth in four years in 2023, with a rebound in its consumer segment and income from new businesses like smart car components accelerating its recovery from U.S. sanctions. At a launch event last year, Meng Wangzhou, Huawei's CFO 

In [9]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
news_string = '\n'.join(news_paras)

# Input text to be summarized
input_text = news_string

# Parse the input text
parser = PlaintextParser.from_string(input_text, Tokenizer("english"))

# Create an LSA summarizer
summarizer = LsaSummarizer()

# Generate the summary
summary = summarizer(parser.document, sentences_count=3)  # You can adjust the number of sentences in the summary

# Output the summary
print("Original Text:")
print(input_text)
print("\nSummary:")
for sentence in summary:
    print(sentence)

Original Text:
China's Huawei Technologies saw its fastest growth in four years in 2023, with a rebound in its consumer segment and income from new businesses like smart car components accelerating its recovery from U.S. sanctions.
Revenue rose 9.63% from a year earlier to 704.2 billion yuan ($97.48 billion), with the consumer business contributing most to that figure, growing 17.3% to 251.49 billion yuan.
While Huawei did not break down the consumer figure, the segment includes its handset business, which saw a renaissance last year as the company returned to the mainstream 5G smartphones market with the Mate 60, apparently overcoming U.S. restrictions.
From 2019, the U.S. has restricted Huawei's access to U.S. technology, accusing the company of being a security risk, which Huawei denies.
Last year marked the third consecutive year of growth for the company after revenue plummeted by almost a third in 2021 when the company started to exhaust chip reserves, though revenue remains belo

In [10]:
news_string

'China\'s Huawei Technologies saw its fastest growth in four years in 2023, with a rebound in its consumer segment and income from new businesses like smart car components accelerating its recovery from U.S. sanctions.\nRevenue rose 9.63% from a year earlier to 704.2 billion yuan ($97.48 billion), with the consumer business contributing most to that figure, growing 17.3% to 251.49 billion yuan.\nWhile Huawei did not break down the consumer figure, the segment includes its handset business, which saw a renaissance last year as the company returned to the mainstream 5G smartphones market with the Mate 60, apparently overcoming U.S. restrictions.\nFrom 2019, the U.S. has restricted Huawei\'s access to U.S. technology, accusing the company of being a security risk, which Huawei denies.\nLast year marked the third consecutive year of growth for the company after revenue plummeted by almost a third in 2021 when the company started to exhaust chip reserves, though revenue remains below its 20

In [11]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv('API_KEY')
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-pro')
response = model.generate_content('summarise this news article in short but dont miss any point: ' + news_string)

print("Summary:")
print(response.text)


/Users/abhasjaiswal/Desktop/News-Aggregator/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Summary:
Huawei achieved its fastest growth in four years in 2023, driven by a rebound in consumer demand, notably in its 5G smartphone segment. The consumer business contributed significantly to the 9.63% revenue increase, which reached 704.2 billion yuan. Despite ongoing US sanctions, Huawei also experienced growth in its cloud and smart car components businesses, resulting in a significant 144.5% increase in net profit to 87 billion yuan.
